# TP - Correction de la distorsion

**Guillaume Bourmaud - guillaume.bourmaud@enseirb-matmeca.fr**

## Utilisation IDE Spyder sur un ordinateur de l'Enseirb
1. Lancer une session linux (et non pas windows)
2. Aller dans "Applications", puis "Autre", puis "conda_pytorch" (un terminal devrait s'ouvrir)
3. Dans ce terminal, taper la commande suivante pour lancer Spyder :  
`spyder &`  
4. Configurer Spyder en suivant ces instructions [Lien configuration Spyder](https://gbourmaud.github.io/files/configuration_spyder_annotated.pdf).
***

L'objectif de ce TP est de corriger la distorsion, vis-à-vis du modèle sténopé, d'une image prise par une caméra. L'image corrigée sera alors considérée idéale du point de vue du modèle sténopé. **Ce TP peut donc être vu comme une simple application d'une transformation à une image.**  

La technique permettant d'appliquer une transformation à une image vous est rappelée dans ce document [transformation_image.pdf](https://gbourmaud.github.io/files/vision/TP/Tutoriel_transformation_image/transformation_image.pdf). Un tutoriel python est disponible ici : [tutoriel_rotation_image.ipynb](https://github.com/gbourmaud/gbourmaud.github.io/blob/master/files/vision/TP/Tutoriel_transformation_image/tutoriel_rotation_image.ipynb)

## Quelle transformation appliquer pour corriger la distorsion ?  
  
  
Dans ce TP, nous allons corriger l'[image suivante](https://gbourmaud.github.io/files/vision/TP/TP_undistortion/videoframe-3.bmp) $\mathbf{I}_{\text{real}}$ issue d'une caméra de type GoPro:  
  
![alt text](videoframe-3.bmp)

L'image distordue $\mathbf{I}_{\text{real}}$ (ci-dessus) n'est pas idéale du point de vue du modèle sténopé. On remarque par exemple que les lignes droites en 3D (comme le bord de la porte, le bord de l'écran, ou encore le bord du tableau) ne sont pas droites dans l'image.  
  
Nous considérons, dans ce TP, le cas où la caméra ayant capturé $\mathbf{I}_{\text{real}}$, a été calibrée. Ainsi un modèle de distorsion de la forme suivante est fourni :  
  
$$\underline{\mathbf{m}}_{\text{real}} = d(\underline{\mathbf{m}}_{\text{ideal}},\mathbf{k})$$  
  
  
où $d(\cdot)$ est une fonction de distorsion, $\mathbf{k}$ est un vecteur de paramètres, $\underline{\mathbf{m}}_{\text{ideal}}$ est un point 2D en coordonnées homogènes du plan focal normalisé idéal et $\underline{\mathbf{m}}_{\text{real}}$ est un point 2D en coordonnées homogènes du plan focal normalisé réel. Dans le cas de notre caméra, le code de la fonction de distorsion est donné ci-après.

In [4]:
def distortion(m_focal_ideal, k):
    #inputs : m_focal_ideal (homogeneous point coordinates in ideal focal plane,  H x W x 3 matrix), k (model parameters)
    #outputs : m_focal_real (homogeneous point coordinates in distorted focal plane,  H x W x 3 matrix)

    kc = k[0]
    xi = k[1]

    X = (1/(m_focal_ideal[:,:,2:3]+xi*np.sqrt(m_focal_ideal[:,:,0:1]**2+m_focal_ideal[:,:,1:2]**2+m_focal_ideal[:,:,2:3]**2)))*m_focal_ideal[:,:,0:2]#space to nplane

    k1 = kc[0]
    k2 = kc[1]
    k3 = kc[2]
    k4 = kc[3]
    k5 = kc[4]    
    
    m_focal_real = np.zeros_like(m_focal_ideal)
 
    x=X[:,:,0]
    y=X[:,:,1]
    r2=x**2+y**2
    radDist = 1. + k1*r2 + k2*(r2**2) + k5*(r2**3)
    m_focal_real[:,:,0] = x*radDist + 2*k3*x*y + k4*(r2+2*(x**2))
    m_focal_real[:,:,1] = y*radDist + k3*(r2+2*(y**2)) + 2*k4*x*y;
    m_focal_real[:,:,2] = 1.
    
    return m_focal_real

Pour notre caméra, le vecteur de paramètres $\mathbf{k}$ contient 6 valeurs que nous découperons en un vecteur de 5 valeurs et un scalaire.

In [ ]:
#%% Camera calibration parameters
k = []
k.append(np.array([-0.616031774058559, 0.236026168622863, -0.0109419992705452, -0.00217955565809950, 0]))
k.append(1.745612606223418)

La matrice de calibration linéaire $\mathbf{K}_{\text{real}}$ est également fournie :

In [ ]:
K_real = np.array([[1.825099190841841e+03, 0., 6.486113006422010e+02],[0., 1.817526262377727e+03, 4.911359689139596e+02],[0., 0., 1.]])

Par conséquent, un point $\underline{\mathbf{m}}_{\text{ideal}}$ du plan focal idéal peut être transformé en un point du plan image réel de la manière suivante :  

$$ \underline{\mathbf{p}}_{\text{real}}=\mathbf{K}_{\text{real}}d(\underline{\mathbf{m}}_{\text{ideal}},\mathbf{k})$$  


Afin d'avoir une transformation entre le plan image idéal et le plan image réel, il nous faut choisir la taille de l'image souhaitée ainsi qu'une matrice de calibration $\mathbf{K}_{\text{ideal}}$. On pourra commencer par utiliser les paramètres suivants :

In [ ]:
#%% Ideal parameters
K_ideal = np.array([[480, 0, 825],[0, 480, 460], [0, 0, 1]])
h_ideal = 900
w_ideal = 1600

Nous disposons alors d'une transformation entre le plan image idéal et le plan image réel :  

$$ \underline{\mathbf{p}}_{\text{real}}=\mathbf{K}_{\text{real}}d(\mathbf{K}_{\text{ideal}}^{-1}\underline{\mathbf{p}}_{\text{ideal}},\mathbf{k})$$  

**Afin d'effectuer la correction de la distorsion, il suffit donc d'appliquer la transformation ci-dessus à $\mathbf{I}_{\text{real}}$ pour obtenir l'image corrigée $\mathbf{I}_{\text{ideal}}$.** Pour ce faire, il est recommandé de partir du code de ce tutoriel [tutoriel_rotation_image.ipynb](https://github.com/gbourmaud/gbourmaud.github.io/blob/master/files/vision/TP/Tutoriel_transformation_image/tutoriel_rotation_image.ipynb) et de le modifier. Il faudra notamment modifier la fonction `getInterpolationGrids` pour que la transformation ci-dessus soit appliquée, au lieu d'un rotation comme c'est le cas dans le tutoriel.  
**SOLUTION : vous trouverez à la fin de ce fichier un exemple d'implémentation de la fonction `getInterpolationGrids` ainsi que le script complet auquels vous pourrez vous référer si vous êtes bloqués.**  

Vous devriez alors obtenir le résultat suivant   
![alt text](im_undist.bmp)

Remarquons que les lignes droites en 3D (bord de la porte, ligne au plafond, etc.) sont désormais également droites dans l'image corrigée. Remarquons également que l'image corrigée a perdu de l'information au niveau des bords. Ceci est dû au choix des paramètres de $\mathbf{K}_{\text{ideal}}$, en particulier la distance focale. Vous pouvez modifier ces paramètres et observer leur impact sur l'image corrigée. Par exemple, diviser par deux la distance focale revient à dézoomer sur le plan focal normalisé idéal. On obtient alors plus d'information (mais des zones noires apparaissent si l'information n'est pas présente dans l'image d'origine) :  
![alt text](im_undist_small_focal.bmp)

## Fonction `remap` d'OpenCV

La fonction [`remap`](https://docs.opencv.org/3.4/d1/da0/tutorial_remap.html) de la bibliothèque OpenCV (`import cv2`) est une fonction d'interpolation 2D très efficace. Cette fonction peut être utilisée pour remplacer la fonction `interp2_bilinear` (disponible dans le code du tutoriel sur la [transformation d'image](https://github.com/gbourmaud/gbourmaud.github.io/blob/master/files/vision/TP/Tutoriel_transformation_image/tutoriel_rotation_image.ipynb)). Tester cette fonction (attention il faudra probalement transformer vos variables, qui sont en float 64, en float 32 de la manière suivante : `.astype(np.float32)`). Vous devriez constater que `remap` est beaucoup plus rapide que la fonction `interp2_bilinear`. En revanche, en comparant les deux images corrigées, vous devriez constater que `remap` introduit de légers artefacts aux endroits où il y a un fort contraste.

### (Optionnel) Calcul du champ de vision

Trouver une manière de calculer de champ de vision (angle $\theta_x$ horizontal et angle $\theta_y$ vertical) de l'image corrigée. Indice pour calculer $\theta_x$ : faire un schéma représentant le repère de la caméra vu de dessus (on a donc deux axes : $x$ et $z$), le plan focal normalisé (sous la forme d'une ligne car il est vu de dessus), les bords gauche et droit de l'image dans le plan focal normalisé et $\theta_x$. Le calcul de $\theta_x$ fait intervenir la fonction `arc tangente`. 

### CORRECTION `getInterpolationGrids.py`

In [ ]:
import numpy as np
from distortion import distortion

def getInterpolationGrids(K_reel, k_reel, K_ideal, h_ideal, w_ideal):

    x = np.arange(0,w_ideal)
    y = np.arange(0,h_ideal)
    X, Y = np.meshgrid(x,y) #h_ideal x w_ideal
    
    p_ideal = np.ones((h_ideal,w_ideal,3)) #h_ideal x w_ideal x 3
    p_ideal[:,:,0] = X
    p_ideal[:,:,1] = Y
    
    K_ideal_inv = np.linalg.inv(K_ideal) #3x3
    
    m_focal_ideal = p_ideal @ (K_ideal_inv.T) #h_ideal x w_ideal x 3

    m_focal_distordu = distortion(m_focal_ideal, k_reel) #h_ideal x w_ideal x 3

    p_image_reel = m_focal_distordu @ (K_reel.T) #h_ideal x w_ideal x 3

    XI = p_image_reel[:,:,0] #h_ideal x w_ideal
    YI = p_image_reel[:,:,1] #h_ideal x w_ideal
 

    return XI, YI

### CORRECTION SCRIPT COMPLET

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from getInterpolationGrids import getInterpolationGrids
from interp2_bilinear import interp2_bilinear
import cv2
import time

#%% Load ideal parameters
K_ideal = np.array([[480, 0, 825],[0, 480, 460], [0, 0, 1]])
h_ideal = 900
w_ideal = 1600

#%% Load camera calibration parameters
K_reel = np.array([[1.825099190841841e+03, 0., 6.486113006422010e+02],[0., 1.817526262377727e+03, 4.911359689139596e+02],[0., 0., 1.]])

k_reel = []
k_reel.append(np.array([-0.616031774058559, 0.236026168622863, -0.0109419992705452, -0.00217955565809950, 0]))
k_reel.append(1.745612606223418)

#%% get interpolation grids

XI,YI = getInterpolationGrids(K_reel, k_reel, K_ideal, h_ideal, w_ideal)

fig1, axs1 = plt.subplots(ncols=2)
axs1[0].imshow(XI)
axs1[1].imshow(YI)
plt.pause(0.1)

#%% load image to undistort
I_reel = np.array(Image.open('videoframe-3.bmp')).astype(float)/255.

fig2, axs2 = plt.subplots(ncols=2)
axs2[0].imshow(I_reel)
plt.pause(0.1)

#%% undistort image

a = time.time()
I_undist,_ = interp2_bilinear(I_reel.astype(np.float32), XI.astype(np.float32), YI.astype(np.float32))
print('{} sec'.format(time.time()-a))

axs2[1].imshow(I_undist)
plt.pause(0.1)

Image.fromarray((I_undist*255).astype(np.uint8)).save("im_undist.bmp")


#%%Comparison with opencv remap
a = time.time()
I_undist_cv2 = cv2.remap(I_reel.astype(np.float32), XI.astype(np.float32), YI.astype(np.float32), cv2.INTER_LINEAR) #very fast but inaccurate
print('{} sec'.format(time.time()-a))

#assert(np.allclose(I_undist, I_undist_cv2, rtol=1e-05))


fig3, axs3 = plt.subplots(ncols=3)
axs3[0].imshow(I_undist)
axs3[0].set_title('True bilinear interp')
axs3[1].imshow(I_undist_cv2)
axs3[1].set_title('opencv')
axs3[2].imshow(np.abs(I_undist-I_undist_cv2).sum(axis=2))
axs3[2].set_title('error')


